这个notebook会带领大家学习基本的SQL操作，包括：
* TABLE
* SELECT
* LIMIT
* WHERE
* ORDER BY
* FUNCTION
* GROUP BY

是不是觉得很熟悉？没错，其实Pandas的API也是参考SQL设计的，他们都反映了我们对数据查询的需求

在这个教程中，为了方便，我们将会使用一个简单但完备的数据库，`DuckDB`，为什么不使用诸如MySQL, MariaDB这样的大型数据库是因为他们太复杂了，安装就需要很长时间，各种配置也会花费大家太多的精力

DuckDB的详细信息请查阅：https://duckdb.org DuckDB的Python API请参阅PEP249 Python DB API 2.0:https://www.python.org/dev/peps/pep-0249/

要安装`DuckDB`,只需要

In [1]:
!pip install duckdb

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


要创建DuckDB临时数据库，只需要:

In [2]:
import duckdb

# :Memory:关键字意为在内存中新建临时数据库
con = duckdb.connect(database=':memory:', read_only=False)

我们进行一个简单的查询

In [3]:
con.execute("SELECT 1")
con.fetchdf()

,1
0,1


通过`con.fetchdf()`函数，我们能将查询的结果转化为pandas dataframe并更好的进行可视化

## TABLE
类似于pandas中的dataframe，在SQL数据库系统中，类似Excel的数据结构是TABLE，TABLE也是一个由多行多列组成的二维数据体系，每一列表示一个特定的字段，比如Age，Sex。

要创建table，请使用`CREATE TABLE`语法，要读取来自外部的数据，使用内建的read_csv_auto()函数，请注意：read_csv_auto()函数仅限于`DuckDB`,不同的数据库对于读取外部源数据由不同的方式，请参考具体的API Documentation

In [4]:
con.execute("CREATE TABLE titanic AS SELECT * FROM read_csv_auto('test.csv')")
con.execute("DESCRIBE titanic")
con.fetchdf()

,Field,Type,Null,Key,Default,Extra
0,PassengerId,INTEGER,YES,NaN,NaN,NaN
1,Pclass,INTEGER,YES,NaN,NaN,NaN
2,Name,VARCHAR,YES,NaN,NaN,NaN
3,Sex,VARCHAR,YES,NaN,NaN,NaN
4,Age,DOUBLE,YES,NaN,NaN,NaN
5,SibSp,INTEGER,YES,NaN,NaN,NaN
6,Parch,INTEGER,YES,NaN,NaN,NaN
7,Ticket,VARCHAR,YES,NaN,NaN,NaN
8,Fare,DOUBLE,YES,NaN,NaN,NaN
9,Cabin,VARCHAR,YES,NaN,NaN,NaN


可以看到`CREATE TABLE`的格式是：`CREATE TABLE` + 表名 + `AS` + SELECT语句。SELECT语句在后面很快就会讲到。

而这里我还用到了一个特殊语句：`DESCRIBE`, DESCRIBE的用途是返回具体的字段名，格式，是否为空，是否是Key，是否有默认值等设定，`DESCRIBE`的具体语法是`DESCRIBE` + 表名

那么现在我们的数据库中已经有了一个叫Titanic的表，我们将使用它作为例子进行讲解

## SELECT
SELECT选择用于选择列表中某些具体的字段，具体的语法是`SELECT` + 字段名 + `FROM` + 表名，SELECT语句是整个SQL的核心，95%以上的时间都是在和SELECT打交道，后续的GROUP BY，ORDER BY，WHERE都可以认为是对SELECT的进一步修饰

有一个特殊的字段名，*，这个字段意味着整体，所有，它将会选择所有字段

In [9]:
con.execute("SELECT * FROM titanic")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


上述的例子中我们选择了整个表，因为我们使用了*选择所有字段

In [10]:
con.execute("SELECT Name FROM titanic")
con.fetchdf()

,Name
0,"Kelly, Mr. James"
1,"Wilkes, Mrs. James (Ellen Needs)"
2,"Myles, Mr. Thomas Francis"
3,"Wirz, Mr. Albert"
4,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)"
...,...
413,"Spector, Mr. Woolf"
414,"Oliva y Ocana, Dona. Fermina"
415,"Saether, Mr. Simon Sivertsen"
416,"Ware, Mr. Frederick"


在这个例子中我们选取了所有逝者的名字，因为我们仅仅选取了Name字段

## LIMIT
在上面那个例子里，我们不难发现，SELECT并没有列的截取限制，他会返回所有的列，这对于特别大的表(上百万行)来说，是一个特别开销巨大的工作，尤其是我们仅仅想做一下SQL Debug或者先选取前面的数据看一看有无规律，这时候我们就要使用LIMIT关键字，LIMIT应该被置于SELECT * FROM TABLE 之后而且一定需要给出一个数字指明是LIMIT多少

In [11]:
con.execute("SELECT * FROM titanic LIMIT 5")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


这时候我们发现我们只选取了整个数据库的前5行，这其实起到了pandas中head()函数的作用，多在非正式的查询中使用LIMIt可以大大减少服务器负担

## WHERE
很多时候，我们想对查询的结果进行一些筛选，比如年龄大于60， 男性...这个时候，我们就需要使用WHERE关键字，WHERE关键字应该被放置在`SELECT * FROM TABLE`和`LIMIT 5`之间，以下是一个例子

In [13]:
con.execute("SELECT * FROM titanic WHERE Age >= 60 LIMIT 5")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
1,905,2,"Howard, Mr. Benjamin",male,63.0,1,0,24065,26.0000,NaN,S
2,940,1,"Bucknell, Mrs. William Robert (Emma Eliza Ward)",female,60.0,0,0,11813,76.2917,D15,C
3,961,1,"Fortune, Mrs. Mark (Mary McDougald)",female,60.0,1,4,19950,263.0000,C23 C25 C27,S
4,973,1,"Straus, Mr. Isidor",male,67.0,1,0,PC 17483,221.7792,C55 C57,S


如果我们有多个条件需要匹配，或者有逻辑关系，我们应该使用`AND`, `OR`这类逻辑关系词

In [14]:
con.execute("SELECT * FROM titanic WHERE Age >= 60 AND Sex = 'male'")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
1,905,2,"Howard, Mr. Benjamin",male,63.0,1,0,24065,26.0000,NaN,S
2,973,1,"Straus, Mr. Isidor",male,67.0,1,0,PC 17483,221.7792,C55 C57,S
3,1034,1,"Ryerson, Mr. Arthur Larned",male,61.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C
4,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S
5,1085,2,"Lingane, Mr. John",male,61.0,0,0,235509,12.3500,NaN,Q
6,1128,1,"Warren, Mr. Frank Manley",male,64.0,1,0,110813,75.2500,D37,C


SQL中针对某个字段具备多重匹配的情况(比如说年龄是60或61或62)，准备了`IN`关键字：

In [15]:
con.execute("SELECT * FROM titanic WHERE Age IN (60, 61, 62)")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,940,1,"Bucknell, Mrs. William Robert (Emma Eliza Ward)",female,60.0,0,0,11813,76.2917,D15,C
1,961,1,"Fortune, Mrs. Mark (Mary McDougald)",female,60.0,1,4,19950,263.0000,C23 C25 C27,S
2,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60.0,1,0,24065,26.0000,NaN,S
3,1034,1,"Ryerson, Mr. Arthur Larned",male,61.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C
4,1085,2,"Lingane, Mr. John",male,61.0,0,0,235509,12.3500,NaN,Q
5,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


针对范围取值的情况，准备了`BETWEEN AND`关键字，比如上面的查询可以这么写：

In [16]:
con.execute("SELECT * FROM titanic WHERE Age BETWEEN 60 AND 62")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
1,940,1,"Bucknell, Mrs. William Robert (Emma Eliza Ward)",female,60.0,0,0,11813,76.2917,D15,C
2,961,1,"Fortune, Mrs. Mark (Mary McDougald)",female,60.0,1,4,19950,263.0000,C23 C25 C27,S
3,1034,1,"Ryerson, Mr. Arthur Larned",male,61.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C
4,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S
5,1085,2,"Lingane, Mr. John",male,61.0,0,0,235509,12.3500,NaN,Q
6,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60.0,1,0,24065,26.0000,NaN,S


但是需要注意BETWEEN AND并没有写>=和<=再拼起来快，因为实际上它就是会帮你翻译成两个拼起来hhh，而且这些BETWEEN AND可能并没有被所有数据库支持，如果可以，请使用最简单的SQL语法以获得更好的稳定性

## ORDER BY
在做查询时，我们经常希望我们的数据是有顺序的呈现，比如说我们想要按年龄倒序，按票号排序，这时候SQL为我们准备了`ORDER BY`关键词，`ORDER BY`需要被放置在WHERE和LIMIT之间，请注意顺序。

In [17]:
con.execute("SELECT * FROM titanic WHERE Age >= 50 ORDER BY Age LIMIT 10")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,917,3,"Robins, Mr. Alexander A",male,50.0,1,0,A/5. 3337,14.5000,NaN,S
1,1247,1,"Julian, Mr. Henry Forbes",male,50.0,0,0,113044,26.0000,E60,S
2,1110,1,"Widener, Mrs. George Dunton (Eleanor Elkins)",female,50.0,1,1,113503,211.5000,C80,C
3,1299,1,"Widener, Mr. George Dunton",male,50.0,1,1,113503,211.5000,C80,C
4,922,2,"Louch, Mr. Charles Alexander",male,50.0,1,0,SC/AH 3085,26.0000,NaN,S
5,1283,1,"Lines, Mrs. Ernest H (Elizabeth Lindsey James)",female,51.0,0,1,PC 17592,39.4000,D28,S
6,1023,1,"Gracie, Col. Archibald IV",male,53.0,0,0,113780,28.5000,C51,C
7,1185,1,"Dodge, Dr. Washington",male,53.0,1,1,33638,81.8583,A34,S
8,1116,1,"Candee, Mrs. Edward (Helen Churchill Hungerford)",female,53.0,0,0,PC 17606,27.4458,NaN,C
9,1266,1,"Dodge, Mrs. Washington (Ruth Vidaver)",female,54.0,1,1,33638,81.8583,A34,S


可以看到现在是逐渐变大的顺序，如果我们希望降序，则可以使用`DESC`关键字

In [18]:
con.execute("SELECT * FROM titanic ORDER BY Age DESC")
con.fetchdf()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,988,1,"Cavendish, Mrs. Tyrell William (Julia Florence...",female,76.0,1,0,19877,78.8500,C46,S
1,973,1,"Straus, Mr. Isidor",male,67.0,1,0,PC 17483,221.7792,C55 C57,S
2,1128,1,"Warren, Mr. Frank Manley",male,64.0,1,0,110813,75.2500,D37,C
3,1071,1,"Compton, Mrs. Alexander Taylor (Mary Eliza Ing...",female,64.0,0,2,PC 17756,83.1583,E45,C
4,1197,1,"Crosby, Mrs. Edward Gifford (Catherine Elizabe...",female,64.0,1,1,112901,26.5500,B26,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1272,3,"O'Connor, Mr. Patrick",male,NaN,0,0,366713,7.7500,NaN,Q
414,1025,3,"Thomas, Mr. Charles P",male,NaN,1,0,2621,6.4375,NaN,C
415,1274,3,"Risien, Mrs. Samuel (Emma)",female,NaN,0,0,364498,14.5000,NaN,S
416,983,3,"Pedersen, Mr. Olaf",male,NaN,0,0,345498,7.7750,NaN,S


可以看到现在年龄呈降序排列

## Function
与Pandas非常相似，SQL提供了一系列函数用于常见的数据处理，比如用于平均数的AVG(), 以及MAX(), MIN(), SUM(), 还有用于对数的LOG(), 指数的EXP()...，要使用他们，请将这些函数放置在SELECT的字段位置，并传入字段作为输入。

而且SQL还支持基本的算术运算符

In [21]:
con.execute("SELECT AVG(Age), MAX(Age), MIN(Age), SUM(Age) FROM titanic")
con.fetchdf()

,avg(age),max(age),min(age),sum(age)
0,30.27259,76.0,0.17,10050.5


对于标准差，duckdb分成population和sample两种：

In [22]:
con.execute("SELECT stddev_pop(Age), stddev_samp(Age) FROM titanic")
con.fetchdf()

,stddev_pop(age),stddev_samp(age)
0,14.159836,14.181209


In [23]:
con.execute("SELECT LOG(Age), EXP(Age), Age/PClass FROM titanic")
con.fetchdf()

,log(age),exp(age),age / pclass
0,1.537819,9.619658e+14,11.500000
1,1.672098,2.581313e+20,15.666667
2,1.792392,8.438357e+26,31.000000
3,1.431364,5.320482e+11,9.000000
4,1.342423,3.584913e+09,7.333333
...,...,...,...
413,NaN,NaN,NaN
414,1.591065,8.659340e+16,39.000000
415,1.585461,5.252155e+16,12.833333
416,NaN,NaN,NaN


有一个非常常用的函数是COUNT，它用于计数某个字段的个数，一般用COUNT( * )因为数据库一般是基于行的(行存储和列存储不做要求)，为了解决有多个重复值的问题，SQL提供了一个DISTINCT关键字用于筛选独特的值个数。

In [28]:
con.execute("SELECT COUNT(*), COUNT(DISTINCT Age) FROM titanic")
con.fetchdf()

,count_star(),count(age)
0,418,79


这里我们看到整个数据库共有418行，关于年龄有79个独特值

每一个数据库支持的函数都不尽相同，请查阅具体的文档，比如DuckDB请翻看https://duckdb.org/docs/sql/functions/overview

## Group By
类似于Pandas的.groupby()函数，SQL也提供了一个类似的`GROUP BY`语句用于将表格按照某个字段的取值进行分割。`GROUP BY`字符串应该被放置在`WHERE`和`ORDER BY`之间，例如我们想选择每一个年龄的逝者数量:

In [32]:
con.execute("SELECT Age, COUNT(*) FROM titanic WHERE PClass = 2 GROUP BY Age ORDER BY Age")
con.fetchdf()

,Age,count_star()
0,NaN,5
1,0.92,1
2,1.00,1
3,2.00,1
4,8.00,2
5,12.00,2
6,14.00,1
7,15.00,1
8,17.00,1
9,18.00,3


`GROUP BY`存在一个常见的错误，我们在GROUP BY后经常想选择其他列的值，但是其他列的值只可以使用Aggregate Function聚合函数而不可以直接使用列名(想一想为什么？)

In [30]:
con.execute("SELECT Name, Age FROM titanic GROUP BY Age")
con.fetchdf()

RuntimeError: Binder Error: column must appear in the GROUP BY clause or be used in an aggregate function

正如这个Error所说，我们应该使用aggregate function聚合其他列的值

In [33]:
con.execute("SELECT AVG(PClass), Age FROM titanic GROUP BY Age")
con.fetchdf()

,avg(pclass),Age
0,3.0,34.50
1,3.0,0.33
2,2.0,0.92
3,3.0,0.83
4,3.0,0.17
...,...,...
75,3.0,40.50
76,2.0,32.50
77,3.0,38.50
78,3.0,60.50


这样就没有问题了，请注意：顺序非常重要，整个语句的顺序应该是`SELECT * FROM TABLE WHERE ... AND ... GROUP BY ... ORDER BY ... LIMIT ...` `SELECT -> FROM -> WHERE -> GROUP -> ORDER -> LIMIT`这里的关键词顺序出现任何调换都会直接导致SQL的崩溃，比如说：

In [34]:
con.execute("SELECT Age, COUNT(*) FROM titanic GROUP BY Age WHERE PClass = 2 ORDER BY Age")
con.fetchdf()

RuntimeError: Parser Error: syntax error at or near "WHERE"
LINE 1: ...ge, COUNT(*) FROM titanic GROUP BY Age WHERE PClass = 2 ORDER BY Age
                                                  ^

GROUP BY提前到WHERE之间直接出错，保持顺序就不会有问题，请一定注意顺序

In [35]:
con.execute("SELECT Age, COUNT(*) FROM titanic WHERE PClass = 2 GROUP BY Age ORDER BY Age")
con.fetchdf()

,Age,count_star()
0,NaN,5
1,0.92,1
2,1.00,1
3,2.00,1
4,8.00,2
5,12.00,2
6,14.00,1
7,15.00,1
8,17.00,1
9,18.00,3


好的，那么到这里你应该已经了解了基础的SQL语法，当然还有很多比较高阶的语法我因为时间关系没能教大家，比如说PARTITION BY，比如说HAVING，以及各种好玩的函数，还有权限控制，还有列存储，还有读写锁，希望大家有时间能继续学习Data Science这门有趣的课程，共勉⛽️